In [ ]:
from Modele import Modele
from Optimiseur import Optimiseur
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.load('ckn_mnist.npz')

# Access the arrays in the dataset
images = data['X']
labels = data['y']
#  Binarization of the labels
labels = np.where(labels < 5, -1, 1)

In [ ]:
n=2000

indices = np.random.choice(images.shape[0], n, replace=False)
images_ = images[indices]
labels_ = labels[indices]
# Separe the dataset into training and testing
n_train = int(n*0.8)
train_images = images_[:n_train]
train_labels = labels_[:n_train]
test_images = images_[n_train:]
test_labels = labels_[n_train:]
# Reshape the images to 2D
X_train = train_images.reshape(train_images.shape[0], -1)
X_test = test_images.reshape(test_images.shape[0], -1)
y_train = train_labels.reshape(-1)
y_test = test_labels.reshape(-1)

In [ ]:
modele = Modele(0,30)
optimiseur = Optimiseur(modele)
L,mu = modele.constante_L(X_train)

In [ ]:
modele = Modele(0,0.5)
optimiseur = Optimiseur(modele)
max_iters=5000
target=0.03
criterion='norm'

n_lr= 20
nb_init = 10

train_loss_003_2k = np.zeros((n_lr,nb_init))
test_loss_003_2k = np.zeros((n_lr,nb_init))
norm_003_2k = np.zeros((n_lr,nb_init))
train_accuracy_003_2k = np.zeros((n_lr,nb_init))
test_accuracy_003_2k = np.zeros((n_lr,nb_init))
nb_ite_003_2k = np.zeros((n_lr,nb_init))

for i in range(nb_init):
    init= np.random.randn(X_train.shape[0]) 

    L_003_2k,mu = modele.constante_L(X_train)
    learning_rates = np.linspace(1/L_003_2k,1.99/L_003_2k,n_lr)

    j=0
    for lr in learning_rates:
        alpha, alpha_list = optimiseur.gradient_descent(X_train, y_train, init.copy(),lr, max_iters, target, criterion)
        train_loss_003_2k[j,i] = modele.loss_function(X_train, y_train, alpha)
        test_loss_003_2k[j,i] = modele.test_loss_function(X_train,X_test, y_test,alpha)
        norm_003_2k[j,i] = np.linalg.norm(modele.gradient(X_train, y_train, alpha))
        train_accuracy_003_2k[j,i] = modele.accuracy(X_train,X_train, y_train, alpha)
        test_accuracy_003_2k[j,i] = modele.accuracy(X_train,X_test, y_test, alpha)
        nb_ite_003_2k[j,i] = len(alpha_list)

        j+=1

print("dimensions alphas: (n_lr,nb_init,X_train.shape[0])")

In [ ]:
learning_rates = np.linspace(1,1.99,n_lr)

print(nb_ite_003_2k)

test_accuracy_mean = np.mean(test_accuracy_003_2k[:,:6], axis=1)
test_accuracy_std = np.std(test_accuracy_003_2k[:,:6], axis=1)

ts = np.linspace(1,1.99,n_lr)


plt.figure()
plt.scatter(ts, test_accuracy_mean, label='Test Accuracy')
plt.plot(ts, test_accuracy_mean)
plt.fill_between(ts, test_accuracy_mean - test_accuracy_std, test_accuracy_mean + test_accuracy_std, alpha=0.2)
plt.xlabel('Learning Rate * L')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Learning Rate')
plt.legend()
plt.show()